# Testing lasio on Digital Well Log Files
[lasio](https://github.com/kinverarity1/lasio) is a Python library for parsing LAS files.

LAS stands for Log ASCII Standard. It's a digital well log file. As far as file formats go, it's less structured than something like a JSON. The file is meant to be human readable to the detrement of a better file structure. It's an odd choice given that it's not common practive to read log data directly from the LAS file. People normally look at the graphs. 

In this Notebook, I will
* Open and parse a large amount of LAS files to test how error prone lasio is.
* Analyze the usage frequency of different types of well log tools
* Plot well logs

The well log data used here is publically available [data from the Kansas Geological Survey](http://www.kgs.ku.edu/PRS/Scans/Log_Summary/index.html). I used their data from January to March 2017, which is 418 LAS files (about 860 MB).

## Open and Parse the Files
Parse all the files and see how many lasio can read. 

In [1]:
import os
import lasio
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [5]:
def get_files(path):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(path):
        f.extend(filenames)
    return f

def get_las_file(file_name, path):
    las_path = os.path.join(os.getcwd(), path, file_name)
    return lasio.read(las_path)

def parse_success_all_files():
    results = {}
    for file in get_files('extract/extracted'):
        try:
            las = get_las_file(file, 'extract/extracted')
            result = 'Pass'
        except:
            result = 'Fail'
        results[file] = result
    return results
    
las_file_tests = pd.Series(parse_success_all_files())

In [6]:
las_file_tests[las_file_tests=='Fail']

1046139290.las    Fail
dtype: object

The one file lasio was unable to read appeared to be incorrectly formatted. lasio appears to work. 

In [7]:
las_file_tests.value_counts()

Pass    417
Fail      1
dtype: int64

## Analyze Tool Names
Generally, a well log will have many tools recording data as the tools move through the well. In a LAS file, tools are described by a mnemonic and a description. 

Listing every tool (mnemonic and description) in the LAS files. 

In [8]:
def gen_tool_names():
    for file in las_file_tests[las_file_tests=='Pass'].keys():
        las = get_las_file(file, 'extract/extracted')
        mnemonics = las.curves.keys()
        decriptions = [las.curves[mnemonic]['descr'] for mnemonic in mnemonics]
        curves =  [{'file': file, 'mnemonic': mnemonic, 'descr': descr} 
                   for mnemonic, descr in zip(mnemonics, decriptions)
                  ]
        for curve in curves:
            yield curve
    
tool_names = pd.DataFrame.from_dict(list(gen_tool_names()))

In [9]:
tool_names.head()

,descr,file,mnemonic
0,Depth,1046139243.las,DEPT
1,Annular Borehole Volume,1046139243.las,ABHV
2,CN Selected Porosity,1046139243.las,CNPOR
3,Caliper from Density Tool,1046139243.las,DCAL
4,Density Porosity,1046139243.las,DPOR


List the most common tools, by mnomonic and description

In [10]:
tool_names['mnemonic'].value_counts().head(10)

DEPT    349
GR      314
SP      295
DPOR    277
RHOB    259
DCAL    257
RLL3    255
RILD    253
RILM    253
RHOC    252
Name: mnemonic, dtype: int64

In [11]:
tool_names['descr'].value_counts().head(10)

                             3136
Depth                         274
Gamma Ray                     165
Density Correction             97
DIL Shallow Resistivity        96
DIL Medium Resistivity         95
DIL Spontaneous Potential      95
DIL Deep Resistivity           95
CN Selected Porosity           77
Rxo / Rt                       77
Name: descr, dtype: int64

Per LAS Standards, there are two possible mnemonics for depth: 'DEPTH' and 'DEPT'.

In [12]:
(tool_names.loc[tool_names['mnemonic'].isin(['DEPTH', 'DEPT'])]
 .pivot_table(index='mnemonic', values='file', aggfunc=len, margins=True)
)

mnemonic
DEPT     349.0
DEPTH     68.0
All      417.0
Name: file, dtype: float64

All LAS files in this dataset have a depth component (Although is not required by LAS standards).

Average tools per LAS file (not including depth columns)

In [18]:
np.round(len(tool_names.loc[~tool_names['mnemonic'].isin(['DEPTH', 'DEPT'])]) / len(las_file_tests[las_file_tests=='Pass']))

20.0

### Aggregate Tool Descriptions by Mnemonics
Some descriptions are just an index number, delete those, and delete blanks.

In [19]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
tool_names['descr'].apply(is_number).value_counts()

False    7669
True     1048
Name: descr, dtype: int64

In [20]:
(tool_names['descr'] == '').value_counts()

False    5581
True     3136
Name: descr, dtype: int64

In [21]:
tool_names.loc[tool_names['descr'].apply(is_number), 'descr'] = np.nan
tool_names.loc[tool_names['descr'] == '', 'descr'] = np.nan

In [22]:
tool_names.describe()

,descr,file,mnemonic
count,4533,8717,8717
unique,359,417,281
top,Depth,1046427108.las,DEPT
freq,274,68,349


In [23]:
m_descriptions =(
    pd.DataFrame(
        tool_names.pivot_table(index=['mnemonic', 'descr'], values='file', aggfunc=len)
    )
    .rename(columns={'file': 'count'})
    )
# m_descriptions = m_descriptions[m_descriptions.index.get_level_values(1) != 'nan']
m_descriptions['percent'] = (m_descriptions['count'] / m_descriptions.groupby(level=0)['count'].sum() * 100).round()
# m_descriptions.head(10)
m_descriptions

count  percent
mnemonic descr                                                             
ABHV     Annular Borehole Volume                                10     29.0
         Annular Volume                                         25     71.0
ACCX     X Axis Accelerometer                                    1    100.0
ACCY     Y Axis Accelerometer                                    1    100.0
ACCZ     Z Axis Accelerometer                                    1    100.0
AF10     Array Induction Four Foot Resistivity A10 {F11.4}       3    100.0
AF20     Array Induction Four Foot Resistivity A20 {F11.4}       3    100.0
AF30     Array Induction Four Foot Resistivity A30 {F11.4}       3    100.0
AF60     Array Induction Four Foot Resistivity A60 {F11.4}       3    100.0
AF90     Array Induction Four Foot Resistivity A90 {F11.4}       3    100.0
AFCO60   Array Induction Four Foot Conductivity A60 {F13.4}      2    100.0
AHVT     Annular Volume                                          2    100.0
AMP3FT   AMP3FT Amplitude                                        8    100.0
AMP5FT   AMP5FT Amplitude                                        6    100.0
AMPAVG   Average Sector Amplitude                                4    100.0
AMPMAX   Maximum Sector Amplitude                                4    100.0
AMPMIN   Minimum Sector Amplitude                                4    100.0
AMPS1    AMPS1 Amplitude                                         1    100.0
AMPS2    AMPS2 Amplitude                                         1    100.0
AMPS3    AMPS3 Amplitude                                         1    100.0
AMPS4    AMPS4 Amplitude                                         1    100.0
AMPS5    AMPS5 Amplitude                                         1    100.0
AMPS6    AMPS6 Amplitude                                         1    100.0
AMPS7    AMPS7 Amplitude                                         1    100.0
AMPS8    AMPS8 Amplitude                                         1    100.0
AO10     Array Induction One Foot Resistivity A10 {F11.4}        3    100.0
AO20     Array Induction One Foot Resistivity A20 {F11.4}        3    100.0
AO30     Array Induction One Foot Resistivity A30 {F11.4}        3    100.0
AO60     Array Induction One Foot Resistivity A60 {F11.4}        3    100.0
AO90     Array Induction One Foot Resistivity A90 {F11.4}        3    100.0
...                                                            ...      ...
SSD      Short spaced Density counts                             2     22.0
STIT     Stuck Tool Indicator, Total {F11.4}                     3    100.0
SWDL     SWN Dolomite Porosity                                   3    100.0
SWLS     SWN Limestone Porosity                                  3    100.0
SWNC     SWN Counts                                              3    100.0
SWPOR    SWN Porosity                                            3    100.0
SWSS     SWN Sandstone Porosity                                  3    100.0
TBHV     Borehole Volume                                        35    100.0
TENS     Cable Tension {F11.4}                                   3     16.0
         Tension                                                16     84.0
TNPH     Thermal Neutron Porosity (Ratio Method) in Sele...      3    100.0
TR11     4' Transit Time                                        36    100.0
TR12     6' Transit Time                                        36    100.0
TR21     3' Transit Time                                        36    100.0
TR22     5' Transit Time                                        36    100.0
TT1      Travel Time (Filtered Heavier) (Transmitter 1 -...      2    100.0
TT2      Travel Time (Filtered Heavier) (Transmitter 2 -...      2    100.0
TT3      Travel Time (Filtered Heavier) (Transmitter 1 -...      2    100.0
TT3FT    3FT Travel Time                                         8    100.0
TT4      Travel Time (Filtered Heavier) (Transmitter 2 -...      2    100.0
TT5FT    5FT Travel Time         

## Mnemonics with Multiple Unique Descriptions
The LAS specifications have no required data dictionary for mnemonic codes or how to describe them. This leads to mnomonics and descriptions not matching between LAS files. 

In [24]:
m_descriptions[m_descriptions['percent'] < 100]

count  percent
mnemonic descr                                                             
ABHV     Annular Borehole Volume                                10     29.0
         Annular Volume                                         25     71.0
AT90     Array Induction Two Foot Resistivity A90 {F11.4}        3     60.0
         Array Induction Two Foot Resistivity A90 {F13.4}        2     40.0
AVOL     19 Annular Volume                                       1      2.0
         Annular Volume                                         39     98.0
CALD     Litho Caliper (Diameter)                                6     60.0
         Litho Density Caliper                                   4     40.0
CALM     MEL Calipe                                              1     11.0
         Micro Spherically Focused Tool (MST-DA) Caliper...      6     67.0
         Microlog Caliper                                        2     22.0
CALN     Compensated Neutron Caliper                             4     40.0
         Compensated Neutron Caliper (Diameter)                  6     60.0
CGXT     2 MCG External Temperature                              1      2.0
         MCG External Temperature                               52     98.0
CILD     DIL Deep Conductivity                                  13     57.0
         Deep Induction Conductivity                             4     17.0
         Deep Induction R Conductivity                           6     26.0
CLDC     8 Density Caliper                                       1      2.0
         Density Caliper                                        53     98.0
DCAL     CDL Caliper                                            51     52.0
         Caliper from Density Tool                              45     45.0
         Differential Caliper {F11.4}                            3      3.0
DCOR     9 Density Correction                                    1      2.0
         Density Correction                                     52     98.0
DEN      10 Compensated Density                                  1      2.0
         Compensated Density                                    53     98.0
DEPT     1  DEPTH                                                3      1.0
         1 Logged depth                                          2      1.0
         DEPTH                                                   1      0.0
...                                                            ...      ...
RHOB     Lithology Density (LDP-DA)                              6      6.0
RHOC     CDL Density Correction                                 51     54.0
         Density Correction                                     43     46.0
RTAO     22 Array Ind. One Res Rt                                1      2.0
         Array Ind. One Res Rt                                  53     98.0
RXRT     RX0 Over RT                                             6     29.0
         RXRT                                                   15     71.0
SFL      Shallow Focussed Resistivity                            4     40.0
         Spherically Focused Resistivity, Environmentall...      6     60.0
SMTU     3 DST Uphole Tension                                    1      2.0
         DST Uphole Tension                                     39     98.0
SP       10  SPON POTENTIAL                                      1      1.0
         DIL Spontaneous Potential                              95     73.0
         SP                                                     15     12.0
         Self Potential                                          4      3.0
         Spontaneou                                              1      1.0
         Spontaneous Potential                                   3      2.0
         Spontaneous Potential (Processed, Internally Ca...      6      5.0
         Spontaneous Potential {F11.4}                           3      2.0
         Spontaneous Potential {F13.4}                           2      2.0
SPCG     5 Spontaneous Potential 

## Resistivity Logs
Identify which types of logs measure resistivity.

In [25]:
m_descriptions[['resistivity' in d.lower() for d in m_descriptions.index.get_level_values(1)]]

,,count,percent
mnemonic,descr,,
AF10,Array Induction Four Foot Resistivity A10 {F11.4},3,100.0
AF20,Array Induction Four Foot Resistivity A20 {F11.4},3,100.0
AF30,Array Induction Four Foot Resistivity A30 {F11.4},3,100.0
AF60,Array Induction Four Foot Resistivity A60 {F11.4},3,100.0
AF90,Array Induction Four Foot Resistivity A90 {F11.4},3,100.0
AO10,Array Induction One Foot Resistivity A10 {F11.4},3,100.0
AO20,Array Induction One Foot Resistivity A20 {F11.4},3,100.0
AO30,Array Induction One Foot Resistivity A30 {F11.4},3,100.0
AO60,Array Induction One Foot Resistivity A60 {F11.4},3,100.0


## Plot Dual Induction Restivity and Gamma Ray Logs
These plots are too large to display in this notebook. View these in the /image out/ directory.

In [26]:
def max_plot_depth(las, depth_mnc):
    d = las[depth_mnc].max()
    round_nearest = 500.0
    return np.ceil(d / round_nearest) * round_nearest

def plot_res_gamma(las, file_name):
    # check for 'DEPT' or 'DEPTH'. Will need to have a variable for depth mnonomic or add a curve with the perfered mnonomic
    depth_mnc = las.keys()[0]
    if not depth_mnc in ['DEPT', 'DEPTH']:
        print('log not indexed by depth')
        return
    needed_parameters = ['GR', 'RILD', 'RILM', 'RILD']
    missing_parameters = [param for param in needed_parameters if not param in las.keys()]
    if missing_parameters:
        # print('required fields not found: {}'.format(', '.join(missing_parameters)))
        return
    
    pp.figure(figsize=(10,80))
    pp.suptitle(file_name)
    pp.subplot(1,2,1)
    pp.title('Gamma Ray\nAPI units', y=1.01)
    pp.plot(las['GR'], las[depth_mnc])
    pp.axis(xmin=0, xmax=150, ymin=0, ymax=max_plot_depth(las, depth_mnc))
    pp.gca().invert_yaxis()
    pp.grid(axis='x')
    pp.gca().xaxis.tick_top()

    pp.subplot(1,2,2)
    pp.title('Resistivity\nohm-m', y=1.01)
    pp.semilogx(las['RLL3'], las[depth_mnc], label='DIL Shallow Resistivity')
    pp.semilogx(las['RILM'], las[depth_mnc], label='DIL Medium Resistivity')
    pp.semilogx(las['RILD'], las[depth_mnc], label='DIL Deep Resistivity', alpha=0.7,)
    pp.axis(xmin=0.2, xmax=2000, ymin=0, ymax=max_plot_depth(las, depth_mnc))
    pp.gca().invert_yaxis()
    pp.grid(axis='x', which='major', linewidth=2)
    pp.grid(axis='x', which='minor')
    pp.gca().xaxis.tick_top()
    pp.legend(loc=2)
    
    pp.tight_layout(rect=(0,0,1,0.95))
    pp.savefig('image out/res gamma {}.png'.format(file_name.split('.')[0]))
    pp.close()

In [27]:
def plot_files():
    for file in las_file_tests[las_file_tests=='Pass'].keys()[:30]:
        las = get_las_file(file, 'extract/extracted')
        plot_res_gamma(las, file)
        
plot_files()